# ランダムフォレスト作成

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from mh_tree_algorithm import DecisionTreeMH

In [62]:
class RandomForest(object):
    def __init__(self, n_estimators=100, random_state=None):
        self.n_estimators = n_estimators # 推定器の数（作成する決定木の数）
        self.random_state = random_state 
    
    def reselect_samples(self, samples, targets):
        '''
        ブートストラップサンプリング
        　　トレーニングサンプル（学習データ）がm個あるとき、重複を許してm個をランダムに選択する方法
        '''
        num_samples, num_features = samples.shape
        if self.random_state != None:
            np.random.seed(self.random_state)
        
        '''
        0からnum_samplesの区間の数値をランダムに選択した要素数num_samples個の配列を返す。
        重複あり
        　　（例）np.random.randint(1,7,5) = [2, 1, 6, 2, 6]
        '''
        rs_sample_index = np.random.randint(0, num_samples, size=num_samples)
        
        '''
        n個ある特徴量からそれよりも少ないd個の特徴量をランダムに選択する。
        d = √n
        　　√nは小数点を繰り上げて四捨五入する。
        '''
        rs_num_features = int(np.ceil(np.sqrt(num_features)))
        
        feature_set = set(range(num_features))
        
        '''
        permutation関数は、引数の配列をランダムに並べ替える。
          permutation(5)は、permutation(list(range(5)))と同じ動作をする。
        　　(例) permutation(5) = [0, 1, 2, 3, 4] or [3, 2, 4, 1,0] or [2, 0, 1, 4, 3]のようになる。
          ここで d = √n = 2のときに、permutation(5) = [2, 0, 1, 4, 3]だとすると、
          　　permutation(5)[0:d] = [2, 0]となる。
        '''
        rs_feature_set = set(np.random.permutation(num_features)[0:rs_num_features].tolist())
        
        remove_feature_index = [i for i in (feature_set - rs_feature_set)]
        rs_samples = samples[rs_sample_index, :]
        rs_samples[:, remove_feature_index] = 0.0
        rs_targets = targets[rs_sample_index]
        return rs_samples, rs_targets
    
    def fit(self, X_train, y_train):
        self.estimators_ = []
        for i in range(self.n_estimators):
            self.estimators_ .append(DecisionTreeMH(random_state=self.random_state))
            rs_X_train, rs_y_target = self.reselect_samples(X_train, y_train)
            self.estimators_[i].fit(rs_X_train, rs_y_target)
        
        self.calc_feature_importances()
    
    def calc_feature_importances(self):
        self.feature_importances_ = [0.0] * len(self.estimators_[0].feature_importances_)
        for i in range(self.n_estimators):
            self.feature_importances_ += self.estimators_[i].feature_importances_ / self.n_estimators
    
    def predict(self, X_test):
        
        '''
        まずはここの処理で各決定木内で予測を行う。
        '''
        pred = []
        for i in range(self.n_estimators):
            pred.append(self.estimators_[i].predict(X_test).tolist())
        pred = np.array(pred)
        
        '''
        次にX_testの行数分ループを回して各行の目的変数のインデックス番号に該当する各決定木の予測結果をそれぞれ取得する。（target変数(型:配列)のこと）
        　　max関数を使って、各決定木の予測結果の内最も多数を占めた目的変数を最終的な目的変数として設定する。
          　　つまり、最も多数派の予測結果を返す多数決理論をこれで実現している。
        '''
        label = []
        for j in range(X_test.shape[0]):
            target = pred[:, j]
            class_count = {i: len(target[target == i]) for i in np.unique(target)}
            label.append(max(class_count.items(), key=lambda x: x[1])[0])
        return np.array(label)
    
    def score(self, sample, target):
        return np.sum(self.predict(sample) == target) / float(len(target))

In [3]:
use_feature_index = [2 ,3]
iris = datasets.load_iris()
X = iris.data[:, use_feature_index]
y = iris.target
feature_names = np.array(iris.feature_names)[use_feature_index]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

n_estimators = 50

In [61]:
clf_m = RandomForest(n_estimators=n_estimators, random_state=300)
clf_m.fit(X_train, y_train)
score_m = clf_m.score(X_test, y_test)

(50, 45)
[[2 1 0 ... 2 0 0]
 [2 1 0 ... 2 0 0]
 [2 1 0 ... 2 0 0]
 ...
 [2 1 0 ... 2 0 0]
 [2 1 0 ... 2 0 0]
 [2 1 0 ... 2 0 0]]
0
(50,)
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
1
(50,)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
2
(50,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
3
(50,)
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
4
(50,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
5
(50,)
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
6
(50,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
7
(50,)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 